# LSA(잠재 의미 분석) 실습 예제 - NLP

In [1]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups

dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
#총 문서 갯수
len(documents)

11314

In [2]:
#어떤 토픽의 문서가 있는지 확인
dataset.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [3]:
#간단한 데이터 전처리
news_df = pd.DataFrame({'document': documents})

# 알파벳 이외의 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z#]", " ")

# 길이가 3이하인 문자 제거
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 3]))

# 소문자로 바꾸기
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [4]:
news_df

,document,clean_doc
0,Well i'm not sure about the story nad it did s...,well sure about story seem biased what disagre...
1,"\n\n\n\n\n\n\nYeah, do you expect people to re...",yeah expect people read actually accept hard a...
2,Although I realize that principle is not one o...,although realize that principle your strongest...
3,Notwithstanding all the legitimate fuss about ...,notwithstanding legitimate fuss about this pro...
4,"Well, I will have to change the scoring on my ...",well will have change scoring playoff pool unf...
...,...,...
11309,"Danny Rubenstein, an Israeli journalist, will ...",danny rubenstein israeli journalist will speak...
11310,\n,
11311,\nI agree. Home runs off Clemens are always m...,agree home runs clemens always memorable kinda...
11312,I used HP DeskJet with Orange Micros Grappler ...,used deskjet with orange micros grappler syste...


In [5]:
# 문서 TF-IDF 벡터화(1000개의 문서만)
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', 
max_features= 1000, # 1,000개의 단어만 추출
max_df = 0.5, 
smooth_idf=True)

X = vectorizer.fit_transform(news_df['clean_doc'])

X.shape # DTM의 행렬 크기 반환

(11314, 1000)

In [6]:
X[:5]

<5x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 104 stored elements in Compressed Sparse Row format>

In [7]:
# 뉴스 토픽이 20개!-> Truncated SVD 실행시, 상위 20개의 특이값만 사용
from sklearn.decomposition import TruncatedSVD

# SVD represent documents and terms in vectors 
svd_model = TruncatedSVD(n_components=20, algorithm='randomized', n_iter=100, random_state=122)

svd_model.fit(X)

svd_model.components_.shape

(20, 1000)

In [8]:
#상위 20개 특이값 확인
svd_model.singular_values_

array([17.15952833,  9.93882749,  8.17139855,  7.92032011,  7.62377374,
        7.5257242 ,  7.25096862,  7.00623237,  6.88289372,  6.85602044,
        6.68476301,  6.56045782,  6.52895929,  6.42222944,  6.33939436,
        6.21686249,  6.17477882,  6.09487639,  6.00247117,  5.90654237])

In [9]:
#1000개의 단어 피처 값 받아오기
terms = vectorizer.get_feature_names()
len(terms)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


1000

In [10]:
#주요 단어 확인
n = 8
components = svd_model.components_
for index, topic in enumerate(components):
    print('Topic %d: '%(index + 1), [terms[i] for i in topic.argsort()[: -n - 1: -1]])

Topic 1:  ['just', 'like', 'know', 'people', 'think', 'does', 'good', 'time']
Topic 2:  ['thanks', 'windows', 'card', 'drive', 'mail', 'file', 'advance', 'files']
Topic 3:  ['game', 'team', 'year', 'games', 'drive', 'season', 'good', 'players']
Topic 4:  ['drive', 'scsi', 'disk', 'hard', 'problem', 'drives', 'just', 'card']
Topic 5:  ['drive', 'know', 'thanks', 'does', 'just', 'scsi', 'drives', 'hard']
Topic 6:  ['just', 'like', 'windows', 'know', 'does', 'window', 'file', 'think']
Topic 7:  ['just', 'like', 'mail', 'bike', 'thanks', 'chip', 'space', 'email']
Topic 8:  ['does', 'know', 'chip', 'like', 'card', 'clipper', 'encryption', 'government']
Topic 9:  ['like', 'card', 'sale', 'video', 'offer', 'jesus', 'good', 'price']
Topic 10:  ['like', 'drive', 'file', 'files', 'sounds', 'program', 'window', 'space']
Topic 11:  ['people', 'like', 'thanks', 'card', 'government', 'windows', 'right', 'think']
Topic 12:  ['think', 'good', 'thanks', 'need', 'chip', 'know', 'really', 'bike']
Topic 1

# LDA(잠재 의미 분석) - sklearn 이용
- https://wikidocs.net/40710

In [11]:
from sklearn.decomposition import LatentDirichletAllocation

lda_model = LatentDirichletAllocation(n_components=10,learning_method='online',random_state=777,max_iter=1)
lda_top = lda_model.fit_transform(X)

print(lda_model.components_)
print(lda_model.components_.shape) 

[[ 1.63298459  7.2900724   8.54896182 ...  7.49193635  0.14374901
   1.59009138]
 [ 0.2896584   0.4117951   0.10113168 ...  0.67965637  0.10694658
   0.10144253]
 [ 4.28469739 38.81395323  4.73014492 ...  6.48742032  0.26542125
   0.1500921 ]
 ...
 [ 0.10114017  0.12831726  0.10621085 ...  0.10549643  0.10103415
   0.12082447]
 [ 0.10124113  0.10107427  0.1011614  ...  0.10302748  0.18381415
   0.10150428]
 [12.83266142 35.67976441 17.33997449 ... 96.14763995 27.25201553
  28.27229401]]
(10, 1000)


In [12]:
# 단어 집합. 1,000개의 단어가 저장됨.
terms = vectorizer.get_feature_names()

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])

get_topics(lda_model.components_,terms)

Topic 1: [('jesus', 75.48), ('people', 57.01), ('bible', 49.68), ('christian', 47.54), ('christians', 46.57)]
Topic 2: [('like', 24.43), ('engine', 21.81), ('speed', 20.07), ('just', 19.91), ('miles', 19.39)]
Topic 3: [('windows', 132.29), ('thanks', 109.8), ('card', 105.85), ('drive', 105.1), ('know', 100.25)]
Topic 4: [('sale', 60.64), ('thanks', 56.21), ('mail', 54.39), ('email', 46.98), ('offer', 42.54)]
Topic 5: [('greek', 22.42), ('steve', 20.57), ('disease', 19.66), ('patients', 18.69), ('posting', 17.17)]
Topic 6: [('satellite', 17.07), ('yeah', 16.11), ('david', 15.61), ('year', 15.17), ('article', 14.52)]
Topic 7: [('games', 25.63), ('phone', 17.55), ('soon', 14.11), ('dave', 13.42), ('cars', 10.94)]
Topic 8: [('israel', 60.09), ('israeli', 44.95), ('arab', 25.55), ('jews', 19.82), ('deleted', 18.89)]
Topic 9: [('armenians', 31.64), ('armenian', 30.42), ('turkish', 27.17), ('turkey', 25.24), ('armenia', 15.84)]
Topic 10: [('people', 175.72), ('just', 162.97), ('think', 160.29

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


### LDA : gensim 이용하여 수행
- https://wikidocs.net/30708

In [13]:
#pip install --upgrade gensim

In [15]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [16]:
from nltk.corpus import stopwords
# NLTK로부터 불용어를 받아온다.
stop_words = stopwords.words('english')
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
# 불용어를 제거합니다.

In [17]:
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0
#아래의 결과가 의미하는 것은 두번째 뉴스에서 52라고 dictionray에서 인덱스 된 단어가 1번 나타났다는것을 의미합니다.
# 추후 이를 이용하여, 

[(52, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1)]


In [18]:
import gensim
NUM_TOPICS = 20 # 20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
#passes : 알고리즘의 동작횟수 - 적절히 지정해주면되는 하이퍼 파라미터
topics = ldamodel.print_topics(num_words=4)
#num_words : 출력하고 싶은 단어의 갯수 지정
for topic in topics:
    print(topic)

(0, '0.009*"smokeless" + 0.006*"sweden" + 0.005*"adobe" + 0.004*"latin"')
(1, '0.010*"nrhj" + 0.007*"wwiz" + 0.006*"bxom" + 0.006*"gizw"')
(2, '0.026*"jesus" + 0.012*"bible" + 0.010*"christ" + 0.008*"word"')
(3, '0.018*"printf" + 0.014*"color" + 0.014*"scsi" + 0.009*"widgets"')
(4, '0.016*"government" + 0.015*"president" + 0.011*"state" + 0.007*"states"')
(5, '0.012*"armenian" + 0.011*"israel" + 0.010*"jews" + 0.009*"armenians"')
(6, '0.013*"sale" + 0.010*"condition" + 0.010*"offer" + 0.010*"shipping"')
(7, '0.012*"drive" + 0.012*"windows" + 0.011*"thanks" + 0.011*"system"')
(8, '0.009*"said" + 0.007*"people" + 0.007*"time" + 0.007*"know"')
(9, '0.016*"guns" + 0.010*"crime" + 0.007*"control" + 0.007*"henrik"')
(10, '0.017*"space" + 0.007*"nasa" + 0.006*"research" + 0.005*"university"')
(11, '0.006*"islam" + 0.006*"john" + 0.005*"filename" + 0.005*"water"')
(12, '0.008*"bike" + 0.008*"like" + 0.007*"good" + 0.007*"much"')
(13, '0.011*"health" + 0.008*"medical" + 0.006*"administration" +

In [24]:
print(ldamodel.print_topics())

[(0, '0.009*"smokeless" + 0.006*"sweden" + 0.005*"adobe" + 0.004*"latin" + 0.004*"mask" + 0.004*"denver" + 0.004*"civic" + 0.004*"epoch" + 0.004*"wrist" + 0.004*"amps"'), (1, '0.010*"nrhj" + 0.007*"wwiz" + 0.006*"bxom" + 0.006*"gizw" + 0.005*"tbxn" + 0.005*"bhjn" + 0.004*"bxlt" + 0.004*"wmbxn" + 0.004*"pnei" + 0.004*"nriz"'), (2, '0.026*"jesus" + 0.012*"bible" + 0.010*"christ" + 0.008*"word" + 0.008*"christian" + 0.007*"christians" + 0.006*"paul" + 0.005*"truth" + 0.005*"matthew" + 0.005*"objective"'), (3, '0.018*"printf" + 0.014*"color" + 0.014*"scsi" + 0.009*"widgets" + 0.008*"colors" + 0.008*"display" + 0.007*"cubs" + 0.007*"vesa" + 0.006*"byte" + 0.006*"adaptec"'), (4, '0.016*"government" + 0.015*"president" + 0.011*"state" + 0.007*"states" + 0.007*"federal" + 0.006*"bill" + 0.006*"right" + 0.006*"congress" + 0.006*"court" + 0.006*"rights"'), (5, '0.012*"armenian" + 0.011*"israel" + 0.010*"jews" + 0.009*"armenians" + 0.009*"turkish" + 0.007*"israeli" + 0.006*"people" + 0.006*"turke

- 각 단어 앞에 붙은 수치는 해당 토픽에 대한 기여도를 보여준다. 토픽 은 총 20개를 가지므로 0에서부터 19까지 번호가 할당되어 있다.

# pyLDAvis를 이용한 LDA 시각화

In [19]:
pip install pyLDAvis

     |████████████████████████████████| 1.7 MB 17.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=9daa2b0272fa7b98b09382dfeac7979d3d4882c25197b6de1ec625bd32705fb4
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis


In [20]:
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


- 좌측의 원딜은 각각 20개의 토픽을 나타낸다.
- 각 원과의 거리가 해당 토픽들 사이의 유사성을 나타낸다. 즉 원이 겹친다면 유사한 토픽이라는 의미입니다.
- 주의 : LDA 모델링시 0에서 19까지의 번호가 할당되었다면 위의 시각화는 1에서 20까지의 값을 가진다.

# 문서 별 토픽 분포 보기
- 이게 메인 목적이였음!

In [21]:
#이미 훈련된 LDA 모델인 ldamodel[]에 전체 데이터가 정수 인코딩된 결과를 넣은 후 확인 가능
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(5, 0.33219028), (11, 0.055800326), (13, 0.1635367), (19, 0.43556947)]
1 번째 문서의 topic 비율은 [(9, 0.026561042), (11, 0.027081039), (13, 0.10331168), (15, 0.09416166), (16, 0.04998462), (19, 0.68182683)]
2 번째 문서의 topic 비율은 [(5, 0.31669068), (13, 0.065375224), (14, 0.020007866), (19, 0.585023)]
3 번째 문서의 topic 비율은 [(11, 0.047225658), (12, 0.15404941), (14, 0.4341207), (16, 0.023993084), (19, 0.32941705)]
4 번째 문서의 topic 비율은 [(6, 0.294947), (16, 0.25788352), (19, 0.41568798)]


In [22]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

In [23]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,19.0,0.4356,"[(5, 0.3321565), (11, 0.05579852), (13, 0.1635..."
1,1,19.0,0.6818,"[(9, 0.02656137), (11, 0.02708023), (13, 0.103..."
2,2,19.0,0.5850,"[(5, 0.31666294), (13, 0.06538049), (14, 0.020..."
3,3,14.0,0.4341,"[(11, 0.0472258), (12, 0.15405431), (14, 0.434..."
4,4,19.0,0.4157,"[(6, 0.294947), (16, 0.2578798), (19, 0.415691..."
5,5,19.0,0.4153,"[(2, 0.1389851), (5, 0.051358785), (11, 0.0914..."
6,6,2.0,0.7743,"[(2, 0.7742751), (7, 0.022137765), (12, 0.0609..."
7,7,19.0,0.5984,"[(5, 0.09560872), (6, 0.044503048), (10, 0.049..."
8,8,12.0,0.5293,"[(11, 0.045765333), (12, 0.52934754), (19, 0.4..."
9,9,18.0,0.3424,"[(2, 0.03617903), (8, 0.07375621), (10, 0.0293..."
